In [1]:
#import Pkg; Pkg.add(Pkg.PackageSpec(url="https://github.com/JuliaComputing/JuliaAcademyData.jl"))
using JuliaAcademyData; activate("Parallel_Computing")

 Activating environment at `~/.julia/packages/JuliaAcademyData/II8LR/courses/Parallel_Computing/Project.toml`
   Updating registry at `~/.julia/registries/General`


LoadError: expected package `CredentialsHandler [864e158e]` to be registered

# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can
lead to long discussions, first as to exactly what is being benchmarked and
secondly what explains the differences.  These simple questions can sometimes
get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for
yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT:
[Boxes and registers](https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb)).

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - Julia (built-in)
    - Julia (hand-written)
    - C (hand-written)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
- Towards exploiting parallelism with Julia
    - Allowing for floating point associativity
    - Making use of four cores at once: built-in
    - Making use of four cores at once: hand-written
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [2]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.7911398120911088
 0.028193008704808786
 0.16899172577020272
 0.9337740052199559
 0.5786948452910516
 0.19638412768342106
 0.3485740696706503
 0.23492264723169098
 0.168045458798042
 0.23505044036165823
 0.9354164496201129
 0.9565890897005969
 0.35545640359230934
 ⋮
 0.538690666755131
 0.31610358218567125
 0.5702773283034033
 0.26933376631696104
 0.49559793844805533
 0.1996539453380306
 0.10532746958115813
 0.3007134921234662
 0.43070816575622217
 0.6341440016996436
 0.7925541799494586
 0.47321942311966003

In [3]:
sum(a)

4.9997673970869295e6

The expected result is ~0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

In [4]:
@time sum(a)

  0.004434 seconds (1 allocation: 16 bytes)


4.9997673970869295e6

In [5]:
@time sum(a)

  0.004557 seconds (1 allocation: 16 bytes)


4.9997673970869295e6

In [6]:
@time sum(a)

  0.004294 seconds (1 allocation: 16 bytes)


4.9997673970869295e6

The `@time` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a `BenchmarkTools.jl` package to make benchmarking easy and accurate:

In [7]:
using BenchmarkTools

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1278
┌ Warning: Package BenchmarkTools does not have Logging in its dependencies:
│ - If you have BenchmarkTools checked out for development and have
│   added Logging as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with BenchmarkTools
└ Loading Logging into BenchmarkTools from project dependency, future warnings for BenchmarkTools are suppressed.


In [8]:
@benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.259 ms (0.00% GC)
  median time:      4.903 ms (0.00% GC)
  mean time:        4.922 ms (0.00% GC)
  maximum time:     6.290 ms (0.00% GC)
  --------------
  samples:          1016
  evals/sample:     1

# 1. Julia Built-in

So that's the performance of Julia's built-in sum — but that could be doing any number of tricks to be fast, including not using Julia at all in the first place! Of course, it is indeed written in Julia, but would it perform if we write a naive implementation ourselves?

In [9]:
@which sum(a)

sum(a::AbstractArray; dims) in Base at reducedim.jl:722

Let's save these benchmark results to a dictionary so we can start keeping track of them and comparing them down the line.

In [10]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.392 ms (0.00% GC)
  median time:      4.969 ms (0.00% GC)
  mean time:        5.005 ms (0.00% GC)
  maximum time:     7.535 ms (0.00% GC)
  --------------
  samples:          999
  evals/sample:     1

In [11]:
d = Dict()
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 1 entry:
  "Julia built-in" => 4.39172

# 2. Julia (hand-written)

In [12]:
function mysum(A)
    s = 0.0
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [13]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     10.778 ms (0.00% GC)
  median time:      11.314 ms (0.00% GC)
  mean time:        11.427 ms (0.00% GC)
  maximum time:     14.496 ms (0.00% GC)
  --------------
  samples:          438
  evals/sample:     1

In [14]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "Julia hand-written" => 10.7781
  "Julia built-in"     => 4.39172

So that's about 2x slower than the builtin definition. We'll see why later on.

But first: is this fast?  How would we know?  Let's compare it to some other languages...

#  3. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [15]:
using Libdl
C_code = """
    #include <stddef.h>
    double c_sum(size_t n, double *X) {
        double s = 0.0;
        for (size_t i = 0; i < n; ++i) {
            s += X[i];
        }
        return s;
    }
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [16]:
c_sum(a)

4.999767397086872e6

In [17]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

true

We can now benchmark the C code directly from Julia:

In [18]:
c_bench = @benchmark c_sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     10.649 ms (0.00% GC)
  median time:      11.443 ms (0.00% GC)
  mean time:        11.580 ms (0.00% GC)
  maximum time:     15.900 ms (0.00% GC)
  --------------
  samples:          432
  evals/sample:     1

In [19]:
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any,Any} with 3 entries:
  "C"                  => 10.6486
  "Julia hand-written" => 10.7781
  "Julia built-in"     => 4.39172

# 4. Python's built in `sum`

The `PyCall` package provides a Julia interface to Python:

In [20]:
using PyCall

┌ Info: Precompiling PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0]
└ @ Base loading.jl:1278
┌ Warning: `@get!(dict, key, default)` at /Users/charlottehaley/.julia/packages/MacroTools/AXAbb/src/utils.jl:168 is deprecated, use `get!(()->default, dict, key)` instead.
└ @ Base deprecated.jl:204
┌ Warning: `@get!(dict, key, default)` at /Users/charlottehaley/.julia/packages/MacroTools/AXAbb/src/utils.jl:189 is deprecated, use `get!(()->default, dict, key)` instead.
└ @ Base deprecated.jl:204
ERROR: LoadError: ArgumentError: Package CSTParser [00ebfdb7-1f24-5e51-bd34-a7502290713f] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.

Stacktrace:
 [1] _require(::Base.PkgId) at ./loading.jl:999
 [2] require(::Base.PkgId) at ./loading.jl:928
 [3] require(::Module, ::Symbol) at ./loading.jl:923
 [4] include(::Function, ::Module, ::String) at ./Base.jl:380
 [5] include(::Module, ::String) at ./Base.jl:368
 [6] top-level scope at none:2
 [

LoadError: Failed to precompile PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0] to /Users/charlottehaley/.julia/compiled/v1.5/PyCall/GkzkC_wMfbj.ji.

In [21]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

LoadError: UndefVarError: pybuiltin not defined

In [22]:
pysum(a)

LoadError: UndefVarError: pysum not defined

In [23]:
pysum(a) ≈ sum(a)

LoadError: UndefVarError: pysum not defined

In [24]:
py_list_bench = @benchmark $pysum($a)

LoadError: UndefVarError: pysum not defined

In [25]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

LoadError: UndefVarError: py_list_bench not defined

# 5. Python: `numpy`

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [26]:
using Conda

LoadError: ArgumentError: Package Conda not found in current path:
- Run `import Pkg; Pkg.add("Conda")` to install the Conda package.


In [27]:
# Conda.add("numpy")

In [28]:
numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

LoadError: UndefVarError: pyimport not defined

In [29]:
numpy_sum(a)

LoadError: UndefVarError: numpy_sum not defined

In [30]:
numpy_sum(a) ≈ sum(a)

LoadError: UndefVarError: numpy_sum not defined

In [31]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

LoadError: UndefVarError: py_numpy_bench not defined

# 6. Python, hand-written

In [32]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

LoadError: LoadError: UndefVarError: @py_str not defined
in expression starting at In[32]:1

In [33]:
py_hand = @benchmark $sum_py($a)

LoadError: UndefVarError: sum_py not defined

In [34]:
sum_py(a)

LoadError: UndefVarError: sum_py not defined

In [35]:
sum_py(a) ≈ sum(a)

LoadError: UndefVarError: sum_py not defined

In [36]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

LoadError: UndefVarError: py_hand not defined

# Summary so far

In [37]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia built-in..............4.4
C..........................10.6
Julia hand-written.........10.8


We seem to have three different performance classes here: The numpy and Julia
builtins are leading the pack, followed by the hand-written Julia and C
definitions. Those seem to be about 2x slower.  And then we have the much much
slower Python definitions over 100x slower.

# Exploiting parallelism with Julia

The fact that our hand-written Julia solution was almost an even multiple of
2x slower than the builtin solutions is a big clue: perhaps theres some sort
of 2x parallelism going on here?

(In fairness, there are ways to exploit parallelism in other languages, too,
but for brevity we won't cover them)

# 7. Julia (allowing floating point associativity)

The `for` loop

```julia
for a in A
    s += a
end
```

defines a very strict _order_ to the summation: Julia follows exactly what you
wrote and adds the elements of `A` to the result `s` in the order it iterates.
Since floating point numbers aren't associative, a rearrangement here would
change the answer — and Julia is loathe to give you different answer than
the one you asked for.

You can, however, tell Julia to relax that rule and allow for associativity
with the `@fastmath` macro. This might allow Julia to rearrange the sum in an
advantageous manner.

In [38]:
function mysum_fast(A)
    s = 0.0
    for a in A
        @fastmath s += a
    end
    s
end

mysum_fast (generic function with 1 method)

In [39]:
j_bench_hand_fast = @benchmark mysum_fast($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.149 ms (0.00% GC)
  median time:      4.898 ms (0.00% GC)
  mean time:        4.910 ms (0.00% GC)
  maximum time:     6.348 ms (0.00% GC)
  --------------
  samples:          1018
  evals/sample:     1

In [40]:
mysum_fast(a)

4.999767397086959e6

In [41]:
d["Julia hand-written fast"] = minimum(j_bench_hand_fast.times) / 1e6
d

Dict{Any,Any} with 4 entries:
  "C"                       => 10.6486
  "Julia hand-written"      => 10.7781
  "Julia built-in"          => 4.39172
  "Julia hand-written fast" => 4.14895

# 8. Distributed Julia (built-in)

We can take this one step further: nearly every modern computer these days has
multiple cores. All the above solutions are working one core hard, but all the
others are just sitting by idly. Let's put them to work!

In [42]:
using Distributed
using DistributedArrays
addprocs(4)
#@sync @everywhere workers() include("/opt/julia-1.0/etc/julia/startup.jl") # Needed just for JuliaBox
@everywhere using DistributedArrays

LoadError: ArgumentError: Package DistributedArrays [aaf54ef3-cdf8-58ed-94cc-d582ad619b94] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


In [43]:
adist = distribute(a)
j_bench_dist = @benchmark sum($adist)

LoadError: UndefVarError: distribute not defined

In [44]:
d["Julia 4x built-in"] = minimum(j_bench_dist.times) / 1e6
d

LoadError: UndefVarError: j_bench_dist not defined

# 9. Distributed Julia (hand-written)

Ok, that might be cheating, too — it's again just calling a library
function. Is it possible to write distributed sum ourselves?

In [45]:
function mysum_dist(a::DArray)
    r = Array{Future}(undef, length(procs(a)))
    for (i, id) in enumerate(procs(a))
        r[i] = @spawnat id sum(localpart(a))
    end
    return sum(fetch.(r))
end

LoadError: UndefVarError: DArray not defined

In [46]:
j_bench_hand_dist = @benchmark mysum_dist($adist)

LoadError: UndefVarError: adist not defined

In [47]:
d["Julia 4x hand-written"] = minimum(j_bench_hand_dist.times) / 1e6
d

LoadError: UndefVarError: j_bench_hand_dist not defined

# Overall Summary

In [48]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia hand-written fast.....4.1
Julia built-in..............4.4
C..........................10.6
Julia hand-written.........10.8


# Key take-aways

* Julia allows for serial C-like performance, even with hand-written functions
* Julia allows us to exploit many forms of parallelism to further improve performance. We demonstrated:
    * Single-processor parallelism with SIMD
    * Multi-process parallelism with DistributedArrays
* But there are many other ways to express parallelism, too!